<a href="https://www.kaggle.com/code/tcordeu/march-madness-2024?scriptVersionId=166872758" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.stats import linregress
from tqdm import tqdm

import glob
import lightgbm as lgb
import numpy as np
import optuna as op
import os
import pandas as pd

op.logging.set_verbosity(op.logging.WARNING)

In [2]:
DATA_DIR = '/kaggle/input/march-machine-learning-mania-2024'

In [3]:
CSV = {}

for path in glob.glob(DATA_DIR + "/*.csv"):
    CSV[os.path.basename(path).split('.')[0]] = pd.read_csv(path, encoding='cp1252')

In [4]:
def device():
    from tensorflow.python.client import device_lib

    return 'gpu' if len(device_lib.list_local_devices()) > 2 else 'cpu'

In [5]:
print("Device: {}".format(device()))

2024-03-13 18:28:06.015896: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 18:28:06.016116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 18:28:06.178307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device: cpu


# Build DF

## Feature Engineering

In [6]:
def build_results(gender):
    csv_names = ['NCAATourneyCompactResults', 'RegularSeasonCompactResults']
    csv_names = list(map(lambda x: gender + x, csv_names))
    csvs      = list(map(lambda x: CSV[x], csv_names))
    
    return pd.concat(csvs)

In [7]:
results_m = build_results('M')
results_w = build_results('W')

display(results_m)
display(results_w)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
186547,2024,114,1454,75,1237,70,A,0
186548,2024,114,1455,74,1412,66,A,0
186549,2024,114,1459,91,1359,69,H,0
186550,2024,114,1462,91,1177,58,H,0


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0
...,...,...,...,...,...,...,...,...
130890,2024,114,3409,76,3396,67,A,0
130891,2024,114,3424,63,3129,57,H,0
130892,2024,114,3433,69,3348,59,A,0
130893,2024,114,3453,70,3236,61,A,0


In [8]:
def build_teams(gender):
    teams = CSV["{}Teams".format(gender)].copy()
    teams = teams.drop('TeamName', axis=1)
    teams = teams.set_index('TeamID')
    
    return teams

In [9]:
teams_m = build_teams('M')
teams_w = build_teams('W') # FIXME: Maybe useless since there is no data aside from TeamName.

display(teams_m)
display(teams_w)

,FirstD1Season,LastD1Season
TeamID,,
1101,2014,2024
1102,1985,2024
1103,1985,2024
1104,1985,2024
1105,2000,2024
...,...,...
1474,2023,2024
1475,2023,2024
1476,2023,2024


""
TeamID
3101
3102
3103
3104
3105
...
3474
3475
3476


In [10]:
def calculate_elo(teams, data, initial_rating=2000, k=140, alpha=None):
    '''
    Calculate Elo ratings for each team based on match data.

    Parameters:
    - teams (array-like): Containing Team-IDs.
    - data (pd.DataFrame): DataFrame with all matches in chronological order.
    - initial_rating (float): Initial rating of an unranked team (default: 2000).
    - k (float): K-factor, determining the impact of each match on team ratings (default: 140).
    - alpha (float or None): Tuning parameter for the multiplier for the margin of victory. No multiplier if None.

    Returns: 
    - list: Historical ratings of the winning team (WTeam).
    - list: Historical ratings of the losing team (LTeam).
    '''
    
    # Dictionary to keep track of current ratings for each team
    team_dict = {}
    for team in teams:
        team_dict[team] = initial_rating
        
    # Lists to store ratings for each team in each game
    r1, r2 = [], []
    margin_of_victory = 1

    # Iterate through the game data
    for wteam, lteam, ws, ls  in tqdm(zip(data.WTeamID, data.LTeamID, data.WScore, data.LScore), total=len(data)):
        # Append current ratings for teams to lists
        r1.append(team_dict[wteam])
        r2.append(team_dict[lteam])

        # Calculate expected outcomes based on Elo ratings
        rateW = 1 / (1 + 10 ** ((team_dict[lteam] - team_dict[wteam]) / initial_rating))
        rateL = 1 / (1 + 10 ** ((team_dict[wteam] - team_dict[lteam]) / initial_rating))
        
        if alpha:
            margin_of_victory = (ws - ls)/alpha

        # Update ratings for winning and losing teams
        team_dict[wteam] += k * margin_of_victory * (1 - rateW)
        team_dict[lteam] += k * margin_of_victory * (0 - rateL)

        # Ensure that ratings do not go below 1
        if team_dict[lteam] < 1:
            team_dict[lteam] = 1
        
    return r1, r2

def create_elo_data(teams, data, initial_rating=2000, k=140, alpha=None):
    '''
    Create a DataFrame with summary statistics of Elo ratings for teams based on historical match data.

    Parameters:
    - teams (array-like): Containing Team-IDs.
    - data (pd.DataFrame): DataFrame with all matches in chronological order.
    - initial_rating (float): Initial rating of an unranked team (default: 2000).
    - k (float): K-factor, determining the impact of each match on team ratings (default: 140).

    Returns: 
    - DataFrame: Summary statistics of Elo ratings for teams throughout a season.
    '''
    
    r1, r2 = calculate_elo(teams, data, initial_rating, k, alpha)
    
    # Concatenate arrays vertically
    seasons = np.concatenate([data.Season, data.Season])
    days = np.concatenate([data.DayNum, data.DayNum])
    teams = np.concatenate([data.WTeamID, data.LTeamID])
    tourney = np.concatenate([data.tourney, data.tourney])
    ratings = np.concatenate([r1, r2])
    # Create a DataFrame
    rating_df = pd.DataFrame({
        'Season': seasons,
        'DayNum': days,
        'TeamID': teams,
        'Rating': ratings,
        'Tourney': tourney
    })

    # Sort DataFrame and remove tournament data
    rating_df.sort_values(['TeamID', 'Season', 'DayNum'], inplace=True)
    rating_df = rating_df[rating_df['Tourney'] == 0]
    grouped = rating_df.groupby(['TeamID', 'Season'])
    results = grouped['Rating'].agg(['mean', 'median', 'std', 'min', 'max', 'last'])
    results.columns = ['Rating_Mean', 'Rating_Median', 'Rating_Std', 'Rating_Min', 'Rating_Max', 'Rating_Last']
    results['Rating_Trend'] = grouped.apply(lambda x: linregress(range(len(x)), x['Rating']).slope, include_groups=False)
    results.reset_index(inplace=True)
    
    return results

In [11]:
def build_elo(gender, results, teams):
    csv_names = ['NCAATourneyCompactResults', 'RegularSeasonCompactResults']
    csv_names = list(map(lambda x: gender + x, csv_names))
    csvs      = list(map(lambda x: CSV[x], csv_names))

    tourneys = results.copy()
    tourneys['tourney'] = 0
    tourneys.loc[len(csvs[0]):, 'tourney'] = 1
    tourneys = tourneys.sort_values(['Season', 'DayNum'])
    
    return create_elo_data(teams.reset_index().TeamID, tourneys).drop('Season', axis=1).groupby('TeamID').mean()

In [12]:
elo_m = build_elo('M', results_m, teams_m)
elo_w = build_elo('W', results_w, teams_w)

display(elo_m)
display(elo_w)

100%|██████████| 189003/189003 [00:00<00:00, 260099.62it/s]
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
100%|██████████| 132478/132478 [00:00<00:00, 251662.99it/s]
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + T

,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend
TeamID,,,,,,,
1101,2236.806245,2236.806245,90.261918,2204.893838,2268.718652,2268.718652,127.649628
1102,2678.124707,2686.604784,214.613224,2561.394128,2763.590190,2561.394128,-43.897197
1103,2750.753871,2747.839316,139.884844,2711.145350,2786.167517,2745.404935,-19.813962
1104,3752.543112,3749.593246,59.858238,3704.294710,3804.856073,3804.856073,52.887944
1105,1108.014030,1108.014030,NaN,1108.014030,1108.014030,1108.014030,NaN
...,...,...,...,...,...,...,...
1460,2521.630842,2521.630842,36.888624,2515.109793,2528.151891,2528.151891,52.168392
1461,2996.400014,2993.142619,74.402542,2956.520610,3036.892349,3035.401649,64.841331
1462,3673.402601,3672.555650,55.840404,3635.467714,3712.211378,3709.733564,55.285432


,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend
TeamID,,,,,,,
3101,2082.192135,2082.192135,NaN,2082.192135,2082.192135,2082.192135,NaN
3102,1984.508622,1988.790357,50.106277,1922.191344,2067.621037,1925.042554,-5.185493
3103,2398.798202,2411.812358,76.239688,2320.285649,2447.995900,2396.330833,-9.812588
3104,3343.228764,3303.620011,139.048957,3268.246483,3529.050165,3529.050165,61.916965
3106,1703.142594,1706.371227,54.592245,1687.099354,1725.899390,1687.099354,1.732950
...,...,...,...,...,...,...,...
3460,2693.141781,2685.545374,99.721223,2641.067568,2762.861693,2678.396161,45.100111
3461,2702.648620,2699.856369,82.784696,2658.883327,2745.437872,2678.542297,-26.319625
3462,3642.380183,3641.255421,57.088424,3611.791013,3676.888721,3669.153178,43.484173


In [13]:
def winner(ids):
    id, wId, lId = ids

    return int(id == wId)

def opponent(x):
    winInt, wId, lId = x
    win = not winInt
    
    return wId if win else lId

def score_diff(x):
    winInt, wScore, lScore = x
    win = not winInt
    
    return (wScore - lScore) if win else (lScore - wScore)

def build_season_results(df):
    season_results = df
    season_results['TeamID'] = season_results[['WTeamID', 'LTeamID']].values.tolist()
    season_results = season_results.explode('TeamID')
    season_results['Win'] = season_results[['TeamID', 'WTeamID', 'LTeamID']].apply(winner, axis=1)
    season_results['Defeat'] = season_results['Win'].apply(lambda x: 1 - x)
    season_results['Games'] = season_results['Win'] + season_results['Defeat']
    season_results['ScoreDiff'] = season_results[['Win', 'WScore', 'LScore']].apply(score_diff, axis=1)
    season_results['OTeamID'] = season_results[['Win', 'WTeamID', 'LTeamID']].apply(opponent, axis=1)
    season_results['Home'] = season_results['WLoc'].apply(lambda x: int(x[0] == 'H'))
    season_results = season_results.drop(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT'], axis=1)
    season_results = season_results.groupby(by=['TeamID', 'OTeamID']).sum()
    season_results['WinRatio'] = season_results['Win'] / season_results['Games']
    season_results = season_results.drop(['Win', 'Defeat'], axis=1)

    return season_results

In [14]:
season_results_m = build_season_results(results_m)
season_results_w = build_season_results(results_w)

display(season_results_m)
display(season_results_w)

Games  ScoreDiff  Home  WinRatio
TeamID OTeamID                                  
1101   1102         1         -4     0       1.0
       1115         1         -8     0       1.0
       1116         2         23     2       0.0
       1117         2         -7     2       0.5
       1122         1         -8     0       1.0
...               ...        ...   ...       ...
1478   1384         1        -37     1       1.0
       1437         1         26     1       0.0
       1447         1         23     1       0.0
       1467         2         20     1       0.0
       1476         2        -39     1       1.0

[58612 rows x 4 columns]

Games  ScoreDiff  Home  WinRatio
TeamID OTeamID                                  
3101   3102         1        -36     1       1.0
       3106         1        -11     1       1.0
       3114         1         -5     0       1.0
       3116         2         26     1       0.0
       3117         1        -12     0       1.0
...               ...        ...   ...       ...
3478   3425         1         51     1       0.0
       3433         1         23     1       0.0
       3447         1        -11     1       1.0
       3467         2        -22     1       1.0
       3476         2        -32     1       1.0

[43970 rows x 4 columns]

In [15]:
def build_rpi(results):
    win_pct = results.copy()[['WinRatio']]
    win_pct = win_pct.groupby('TeamID').mean()
    win_pct['WP'] = win_pct['WinRatio'] * 100
    win_pct = win_pct.drop('WinRatio', axis=1)
    
    rpi = results.copy().reset_index()
    rpi = pd.merge(rpi, win_pct, on=['TeamID'])
    rpi = pd.merge(rpi, win_pct, left_on=['OTeamID'], right_on=['TeamID'], suffixes=('_T', '_O'))
    
    wp_oo = rpi[['TeamID', 'WP_O']].groupby('TeamID').mean()
    wp_oo = wp_oo.rename(columns={'WP_O': 'WP_OO'})
    
    rpi = pd.merge(rpi, wp_oo, left_on=['OTeamID'], right_on=['TeamID'])

    rpi['RPI'] = (rpi['WP_T'] * 0.25) + (rpi['WP_O'] * 0.50) + (rpi['WP_OO'] * 0.25)
    
    return rpi[['TeamID', 'OTeamID', 'RPI']].set_index(['TeamID', 'OTeamID'])

In [16]:
rpi_m = build_rpi(season_results_m)
rpi_w = build_rpi(season_results_w)

display(rpi_m)
display(rpi_w)

RPI
TeamID OTeamID           
1101   1102     45.734030
       1115     30.591411
       1116     61.795097
       1117     45.884441
       1122     41.329277
...                   ...
1478   1384     32.692573
       1437     59.590598
       1447     38.022702
       1467     36.846500
       1476     28.612473

[58612 rows x 1 columns]

RPI
TeamID OTeamID           
3101   3102     36.701758
       3106     36.314599
       3114     52.287270
       3116     64.288236
       3117     48.623450
...                   ...
3478   3425     57.041476
       3433     50.098405
       3447     32.217247
       3467     38.531362
       3476     29.282346

[43970 rows x 1 columns]

In [17]:
def clean_seeds(seed):
    res = seed[1:]

    if len(res) > 2:
        res = res[:-1]

    return int(res)

def build_seeds(gender):
    seeds = CSV["{}NCAATourneySeeds".format(gender)] 
    seeds['Seed'] = seeds['Seed'].apply(clean_seeds)
    seeds = seeds.drop('Season', axis=1)
    seeds = seeds.groupby(by='TeamID').mean()
    
    return seeds

In [18]:
seeds_m = build_seeds('M')
seeds_w = build_seeds('W')

display(seeds_m)
display(seeds_w)

,Seed
TeamID,
1101,14.500000
1102,12.000000
1103,13.600000
1104,5.894737
1105,16.000000
...,...
1459,12.200000
1460,15.000000
1461,10.800000


,Seed
TeamID,
3101,16.000000
3103,13.000000
3104,6.000000
3106,15.333333
3107,14.285714
...,...
3458,7.000000
3460,13.333333
3461,12.500000


In [19]:
def build_rankings(gender):
    rankings = CSV["{}MasseyOrdinals".format(gender)]
    rankings = rankings.drop(['SystemName', 'RankingDayNum'], axis=1)
    rankings = rankings.groupby(by='TeamID').mean()
    rankings = rankings.drop('Season', axis=1)

    return rankings

In [20]:
rankings_m = build_rankings('M')

rankings_m

,OrdinalRank
TeamID,
1101,231.982452
1102,183.893379
1103,111.715467
1104,55.569038
1105,319.997840
...,...
1474,227.867379
1475,288.027638
1476,328.846591


In [21]:
def build_history(season_results, seeds, teams, elo, rpi, rankings=None):
    history = season_results.join(teams, on='TeamID').join(seeds, on='TeamID').join(elo, on='TeamID').join(rpi, on=['TeamID', 'OTeamID'])
    history = history.reset_index()
    history = pd.merge(history, rpi.reset_index().rename(columns={'TeamID': 'OTeamID', 'OTeamID': 'TeamID'}), on=['TeamID', 'OTeamID'], suffixes=('_T', '_O'))
    history = pd.merge(history, seeds, left_on='OTeamID', right_on='TeamID', suffixes=('_T', '_O'))
    history['RPIDiff'] = history['RPI_T'] - history['RPI_O']
    history['SeedDiff'] = history['Seed_T'] - history['Seed_O']
    history = history.drop(['RPI_T', 'RPI_O', 'Seed_T', 'Seed_O'], axis=1)

    if rankings is not None:
        history = history.join(rankings, on='TeamID')
        history = pd.merge(history, rankings, left_on='OTeamID', right_on='TeamID', suffixes=('_T', '_O'))
        history['RankingsDiff'] = history['OrdinalRank_T'] - history['OrdinalRank_O']
        history = history.drop(['OrdinalRank_T', 'OrdinalRank_O'], axis=1)
    
    return history.set_index(['TeamID', 'OTeamID']).fillna(0)

In [22]:
history_m = build_history(season_results_m, seeds_m, teams_m, elo_m, rpi_m, rankings_m)
history_w = build_history(season_results_w, seeds_w, teams_w, elo_w, rpi_w)

display(history_m)
display(history_w)

Games  ScoreDiff  Home  WinRatio  FirstD1Season  LastD1Season  \
TeamID OTeamID                                                                  
1101   1102         1         -4     0       1.0           2014          2024   
       1115         1         -8     0       1.0           2014          2024   
       1116         2         23     2       0.0           2014          2024   
       1117         2         -7     2       0.5           2014          2024   
       1122         1         -8     0       1.0           2014          2024   
...               ...        ...   ...       ...            ...           ...   
1478   1360         1          9     1       0.0           2024          2024   
       1364         1         24     1       0.0           2024          2024   
       1384         1        -37     1       1.0           2024          2024   
       1437         1         26     1       0.0           2024          2024   
       1447         1         23     1       0.0           2024          2024   

                Rating_Mean  Rating_Median  Rating_Std   Rating_Min  \
TeamID OTeamID                                                        
1101   1102     2236.806245    2236.806245   90.261918  2204.893838   
       1115     2236.806245    2236.806245   90.261918  2204.893838   
       1116     2236.806245    2236.806245   90.261918  2204.893838   
       1117     2236.806245    2236.806245   90.261918  2204.893838   
       1122     2236.806245    2236.806245   90.261918  2204.893838   
...                     ...            ...         ...          ...   
1478   1360        0.000000       0.000000    0.000000     0.000000   
       1364        0.000000       0.000000    0.000000     0.000000   
       1384        0.000000       0.000000    0.000000     0.000000   
       1437        0.000000       0.000000    0.000000     0.000000   
       1447        0.000000       0.000000    0.000000     0.000000   

                 Rating_Max  Rating_Last  Rating_Trend    RPIDiff  SeedDiff  \
TeamID OTeamID                                                                
1101   1102     2268.718652  2268.718652    127.649628   1.382734   2.50000   
       1115     2268.718652  2268.718652    127.649628  -5.214985  -1.50000   
       1116     2268.718652  2268.718652    127.649628   9.868280   8.23913   
       1117     2268.718652  2268.718652    127.649628   1.778062  -0.50000   
       1122     2268.718652  2268.718652    127.649628  -0.530360   0.10000   
...                     ...          ...           ...        ...       ...   
1478   1360        0.000000     0.000000      0.000000   8.703788   0.00000   
       1364        0.000000     0.000000      0.000000   8.121269   0.00000   
       1384        0.000000     0.000000      0.000000   0.861389   0.00000   
       1437        0.000000     0.000000      0.000000  14.971577   0.00000   
       1447        0.000000     0.000000      0.000000   3.046811   0.00000   

                RankingsDiff  
TeamID OTeamID                
1101   1102        48.089073  
       1115       -85.717845  
       1116       172.308232  
       1117        21.451895  
       1122        11.796366  
...                      ...  
1478   1360       134.127209  
       1364       168.999021  
       1384        45.369540  
       1437       290.879073  
       1447        90.708548  

[51584 rows x 16 columns]

Games  ScoreDiff  Home  WinRatio  Rating_Mean  Rating_Median  \
TeamID OTeamID                                                                 
3101   3106         1        -11     1  1.000000  2082.192135    2082.192135   
       3114         1         -5     0  1.000000  2082.192135    2082.192135   
       3116         2         26     1  0.000000  2082.192135    2082.192135   
       3124         2        102     2  0.000000  2082.192135    2082.192135   
       3146        15        -16     7  0.533333  2082.192135    2082.192135   
...               ...        ...   ...       ...          ...            ...   
3478   3357         2         14     0  0.500000     0.000000       0.000000   
       3384         1          3     1  0.000000     0.000000       0.000000   
       3392         1         21     0  0.000000     0.000000       0.000000   
       3425         1         51     1  0.000000     0.000000       0.000000   
       3433         1         23     1  0.000000     0.000000       0.000000   

                Rating_Std   Rating_Min   Rating_Max  Rating_Last  \
TeamID OTeamID                                                      
3101   3106            0.0  2082.192135  2082.192135  2082.192135   
       3114            0.0  2082.192135  2082.192135  2082.192135   
       3116            0.0  2082.192135  2082.192135  2082.192135   
       3124            0.0  2082.192135  2082.192135  2082.192135   
       3146            0.0  2082.192135  2082.192135  2082.192135   
...                    ...          ...          ...          ...   
3478   3357            0.0     0.000000     0.000000     0.000000   
       3384            0.0     0.000000     0.000000     0.000000   
       3392            0.0     0.000000     0.000000     0.000000   
       3425            0.0     0.000000     0.000000     0.000000   
       3433            0.0     0.000000     0.000000     0.000000   

                Rating_Trend    RPIDiff   SeedDiff  
TeamID OTeamID                                      
3101   3106              0.0  -6.746670   0.666667  
       3114              0.0   0.762601   3.666667  
       3116              0.0   7.371887   8.375000  
       3124              0.0  10.647478  13.333333  
       3146              0.0  -3.264970   2.000000  
...                      ...        ...        ...  
3478   3357              0.0   1.149985   0.000000  
       3384              0.0  -1.107988   0.000000  
       3392              0.0   3.097844   0.000000  
       3425              0.0   9.916086   0.000000  
       3433              0.0   5.491384   0.000000  

[35234 rows x 13 columns]

In [23]:
def build_avg(history):
    agg = {}
    for col in history.columns:
        if col == 'Games' or col == 'Home':
            agg[col] = 'sum'
        else:
            agg[col] = 'mean'
    
    avg = history.groupby('TeamID').agg(agg)
    
    return avg

In [24]:
avg_m = build_avg(history_m)
avg_w = build_avg(history_w)

display(avg_m)
display(avg_w)

,Games,ScoreDiff,Home,WinRatio,FirstD1Season,LastD1Season,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend,RPIDiff,SeedDiff,RankingsDiff
TeamID,,,,,,,,,,,,,,,,
1101,202,9.930556,120,0.346346,2014.0,2024.0,2236.806245,2236.806245,90.261918,2204.893838,2268.718652,2268.718652,127.649628,3.014131,2.842950,65.067294
1102,978,32.950000,566,0.444344,1985.0,2024.0,2678.124707,2686.604784,214.613224,2561.394128,2763.590190,2561.394128,-43.897197,1.377593,0.285921,17.316532
1103,948,-14.933333,590,0.558109,1985.0,2024.0,2750.753871,2747.839316,139.884844,2711.145350,2786.167517,2745.404935,-19.813962,-1.606937,1.760979,-56.697090
1104,1247,-28.252577,725,0.705702,1985.0,2024.0,3752.543112,3749.593246,59.858238,3704.294710,3804.856073,3804.856073,52.887944,-5.086377,-4.722411,-88.669576
1105,596,42.882979,382,0.118832,2000.0,2024.0,1108.014030,1108.014030,0.000000,1108.014030,1108.014030,1108.014030,0.000000,9.695917,4.951648,165.016708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,45,8.185185,32,0.327160,2023.0,2024.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.932835,0.000000,41.101372
1475,31,11.062500,22,0.244792,2023.0,2024.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.605015,0.000000,130.377557
1476,39,18.227273,28,0.162879,2023.0,2024.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.482117,0.000000,147.628040


,Games,ScoreDiff,Home,WinRatio,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend,RPIDiff,SeedDiff
TeamID,,,,,,,,,,,,,
3101,182,-4.764706,102,0.478283,2082.192135,2082.192135,0.000000,2082.192135,2082.192135,2082.192135,0.000000,-0.210233,3.772243
3102,615,98.376344,336,0.202775,1984.508622,1988.790357,50.106277,1922.191344,2067.621037,1925.042554,-5.185493,7.760018,0.000000
3103,677,36.882353,374,0.368098,2398.798202,2411.812358,76.239688,2320.285649,2447.995900,2396.330833,-9.812588,2.386839,1.785072
3104,721,6.717391,374,0.657250,3343.228764,3303.620011,139.048957,3268.246483,3529.050165,3529.050165,61.916965,-3.638035,-4.505748
3105,558,61.821918,345,0.296334,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.171904,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,33,33.166667,20,0.129630,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.978908,0.000000
3475,33,5.214286,18,0.392857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.354288,0.000000
3476,30,25.375000,16,0.145833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.875668,0.000000


In [25]:
def build_matchups(gender):
    teams = CSV["{}Teams".format(gender)].copy()
    teams = teams[['TeamID']]
    teams = pd.merge(teams, teams, how='cross')
    teams = teams.rename(columns={'TeamID_x': 'TeamID', 'TeamID_y': 'OTeamID'})
    teams = teams[teams['TeamID'] != teams['OTeamID']]
    teams = teams.set_index(['TeamID', 'OTeamID'])

    return teams

In [26]:
matchups_m = build_matchups('M')
matchups_w = build_matchups('W')

display(matchups_m)
display(matchups_w)

Empty DataFrame
Columns: []
Index: [(1101, 1102), (1101, 1103), (1101, 1104), (1101, 1105), (1101, 1106), (1101, 1107), (1101, 1108), (1101, 1109), (1101, 1110), (1101, 1111), (1101, 1112), (1101, 1113), (1101, 1114), (1101, 1115), (1101, 1116), (1101, 1117), (1101, 1118), (1101, 1119), (1101, 1120), (1101, 1121), (1101, 1122), (1101, 1123), (1101, 1124), (1101, 1125), (1101, 1126), (1101, 1127), (1101, 1128), (1101, 1129), (1101, 1130), (1101, 1131), (1101, 1132), (1101, 1133), (1101, 1134), (1101, 1135), (1101, 1136), (1101, 1137), (1101, 1138), (1101, 1139), (1101, 1140), (1101, 1141), (1101, 1142), (1101, 1143), (1101, 1144), (1101, 1145), (1101, 1146), (1101, 1147), (1101, 1148), (1101, 1149), (1101, 1150), (1101, 1151), (1101, 1152), (1101, 1153), (1101, 1154), (1101, 1155), (1101, 1156), (1101, 1157), (1101, 1158), (1101, 1159), (1101, 1160), (1101, 1161), (1101, 1162), (1101, 1163), (1101, 1164), (1101, 1165), (1101, 1166), (1101, 1167), (1101, 1168), (1101, 1169), (1101, 1170), (1101, 1171), (1101, 1172), (1101, 1173), (1101, 1174), (1101, 1175), (1101, 1176), (1101, 1177), (1101, 1178), (1101, 1179), (1101, 1180), (1101, 1181), (1101, 1182), (1101, 1183), (1101, 1184), (1101, 1185), (1101, 1186), (1101, 1187), (1101, 1188), (1101, 1189), (1101, 1190), (1101, 1191), (1101, 1192), (1101, 1193), (1101, 1194), (1101, 1195), (1101, 1196), (1101, 1197), (1101, 1198), (1101, 1199), (1101, 1200), (1101, 1201), ...]

[142506 rows x 0 columns]

Empty DataFrame
Columns: []
Index: [(3101, 3102), (3101, 3103), (3101, 3104), (3101, 3105), (3101, 3106), (3101, 3107), (3101, 3108), (3101, 3109), (3101, 3110), (3101, 3111), (3101, 3112), (3101, 3113), (3101, 3114), (3101, 3115), (3101, 3116), (3101, 3117), (3101, 3118), (3101, 3119), (3101, 3120), (3101, 3121), (3101, 3122), (3101, 3123), (3101, 3124), (3101, 3125), (3101, 3126), (3101, 3127), (3101, 3128), (3101, 3129), (3101, 3130), (3101, 3131), (3101, 3132), (3101, 3133), (3101, 3134), (3101, 3135), (3101, 3136), (3101, 3137), (3101, 3138), (3101, 3139), (3101, 3140), (3101, 3141), (3101, 3142), (3101, 3143), (3101, 3144), (3101, 3145), (3101, 3146), (3101, 3147), (3101, 3148), (3101, 3149), (3101, 3150), (3101, 3151), (3101, 3152), (3101, 3153), (3101, 3155), (3101, 3156), (3101, 3157), (3101, 3158), (3101, 3159), (3101, 3160), (3101, 3161), (3101, 3162), (3101, 3163), (3101, 3164), (3101, 3165), (3101, 3166), (3101, 3167), (3101, 3168), (3101, 3169), (3101, 3170), (3101, 3171), (3101, 3172), (3101, 3173), (3101, 3174), (3101, 3175), (3101, 3176), (3101, 3177), (3101, 3178), (3101, 3179), (3101, 3180), (3101, 3181), (3101, 3182), (3101, 3183), (3101, 3184), (3101, 3185), (3101, 3186), (3101, 3187), (3101, 3188), (3101, 3189), (3101, 3190), (3101, 3191), (3101, 3192), (3101, 3193), (3101, 3194), (3101, 3195), (3101, 3196), (3101, 3197), (3101, 3198), (3101, 3199), (3101, 3200), (3101, 3201), (3101, 3202), ...]

[141000 rows x 0 columns]

In [27]:
def build_df(history, matchups, avg):
    df = pd.merge(matchups, history, on=['TeamID', 'OTeamID'], how='left')
    df = df.fillna(avg).fillna(0)

    if 'FirstD1Season' in df.columns:
        df['FirstD1Season'] = df['FirstD1Season'].astype(int)
        df['LastD1Season'] = df['LastD1Season'].astype(int)
    
    return df

In [28]:
df_m = build_df(history_m, matchups_m, avg_m)
df_w = build_df(history_w, matchups_w, avg_w)

display(df_m)
display(df_w)

Games  ScoreDiff   Home  WinRatio  FirstD1Season  \
TeamID OTeamID                                                     
1101   1102       1.0  -4.000000    0.0  1.000000           2014   
       1103     202.0   9.930556  120.0  0.346346           2014   
       1104     202.0   9.930556  120.0  0.346346           2014   
       1105     202.0   9.930556  120.0  0.346346           2014   
       1106     202.0   9.930556  120.0  0.346346           2014   
...               ...        ...    ...       ...            ...   
1478   1473      17.0   7.142857   13.0  0.285714           2024   
       1474      17.0   7.142857   13.0  0.285714           2024   
       1475      17.0   7.142857   13.0  0.285714           2024   
       1476      17.0   7.142857   13.0  0.285714           2024   
       1477      17.0   7.142857   13.0  0.285714           2024   

                LastD1Season  Rating_Mean  Rating_Median  Rating_Std  \
TeamID OTeamID                                                         
1101   1102             2024  2236.806245    2236.806245   90.261918   
       1103             2024  2236.806245    2236.806245   90.261918   
       1104             2024  2236.806245    2236.806245   90.261918   
       1105             2024  2236.806245    2236.806245   90.261918   
       1106             2024  2236.806245    2236.806245   90.261918   
...                      ...          ...            ...         ...   
1478   1473             2024     0.000000       0.000000    0.000000   
       1474             2024     0.000000       0.000000    0.000000   
       1475             2024     0.000000       0.000000    0.000000   
       1476             2024     0.000000       0.000000    0.000000   
       1477             2024     0.000000       0.000000    0.000000   

                 Rating_Min   Rating_Max  Rating_Last  Rating_Trend   RPIDiff  \
TeamID OTeamID                                                                  
1101   1102     2204.893838  2268.718652  2268.718652    127.649628  1.382734   
       1103     2204.893838  2268.718652  2268.718652    127.649628  3.014131   
       1104     2204.893838  2268.718652  2268.718652    127.649628  3.014131   
       1105     2204.893838  2268.718652  2268.718652    127.649628  3.014131   
       1106     2204.893838  2268.718652  2268.718652    127.649628  3.014131   
...                     ...          ...          ...           ...       ...   
1478   1473        0.000000     0.000000     0.000000      0.000000  5.429503   
       1474        0.000000     0.000000     0.000000      0.000000  5.429503   
       1475        0.000000     0.000000     0.000000      0.000000  5.429503   
       1476        0.000000     0.000000     0.000000      0.000000  5.429503   
       1477        0.000000     0.000000     0.000000      0.000000  5.429503   

                SeedDiff  RankingsDiff  
TeamID OTeamID                          
1101   1102      2.50000     48.089073  
       1103      2.84295     65.067294  
       1104      2.84295     65.067294  
       1105      2.84295     65.067294  
       1106      2.84295     65.067294  
...                  ...           ...  
1478   1473      0.00000    121.216628  
       1474      0.00000    121.216628  
       1475      0.00000    121.216628  
       1476      0.00000    121.216628  
       1477      0.00000    121.216628  

[142506 rows x 16 columns]

Games  ScoreDiff   Home  WinRatio  Rating_Mean  Rating_Median  \
TeamID OTeamID                                                                  
3101   3102     182.0  -4.764706  102.0  0.478283  2082.192135    2082.192135   
       3103     182.0  -4.764706  102.0  0.478283  2082.192135    2082.192135   
       3104     182.0  -4.764706  102.0  0.478283  2082.192135    2082.192135   
       3105     182.0  -4.764706  102.0  0.478283  2082.192135    2082.192135   
       3106       1.0 -11.000000    1.0  1.000000  2082.192135    2082.192135   
...               ...        ...    ...       ...          ...            ...   
3478   3473      16.0  16.500000   10.0  0.178571     0.000000       0.000000   
       3474      16.0  16.500000   10.0  0.178571     0.000000       0.000000   
       3475      16.0  16.500000   10.0  0.178571     0.000000       0.000000   
       3476      16.0  16.500000   10.0  0.178571     0.000000       0.000000   
       3477      16.0  16.500000   10.0  0.178571     0.000000       0.000000   

                Rating_Std   Rating_Min   Rating_Max  Rating_Last  \
TeamID OTeamID                                                      
3101   3102            0.0  2082.192135  2082.192135  2082.192135   
       3103            0.0  2082.192135  2082.192135  2082.192135   
       3104            0.0  2082.192135  2082.192135  2082.192135   
       3105            0.0  2082.192135  2082.192135  2082.192135   
       3106            0.0  2082.192135  2082.192135  2082.192135   
...                    ...          ...          ...          ...   
3478   3473            0.0     0.000000     0.000000     0.000000   
       3474            0.0     0.000000     0.000000     0.000000   
       3475            0.0     0.000000     0.000000     0.000000   
       3476            0.0     0.000000     0.000000     0.000000   
       3477            0.0     0.000000     0.000000     0.000000   

                Rating_Trend   RPIDiff  SeedDiff  
TeamID OTeamID                                    
3101   3102              0.0 -0.210233  3.772243  
       3103              0.0 -0.210233  3.772243  
       3104              0.0 -0.210233  3.772243  
       3105              0.0 -0.210233  3.772243  
       3106              0.0 -6.746670  0.666667  
...                      ...       ...       ...  
3478   3473              0.0  4.228141  0.000000  
       3474              0.0  4.228141  0.000000  
       3475              0.0  4.228141  0.000000  
       3476              0.0  4.228141  0.000000  
       3477              0.0  4.228141  0.000000  

[141000 rows x 13 columns]

## Feature analysis

In [29]:
corr_m = df_m.corr()
corr_m.style.background_gradient(cmap='coolwarm')

,Games,ScoreDiff,Home,WinRatio,FirstD1Season,LastD1Season,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend,RPIDiff,SeedDiff,RankingsDiff
Games,1.000000,-0.037826,0.996534,0.093182,-0.287709,0.165160,0.223193,0.223157,0.145506,0.220241,0.225322,0.221359,0.090082,-0.120625,-0.034956,-0.093288
ScoreDiff,-0.037826,1.000000,-0.021034,-0.491599,0.085927,-0.056504,-0.340080,-0.340599,0.087409,-0.348050,-0.329708,-0.344092,-0.234079,0.502092,0.398305,0.497197
Home,0.996534,-0.021034,1.000000,0.073625,-0.281736,0.162693,0.198064,0.197989,0.150966,0.194694,0.200830,0.195903,0.072973,-0.095184,-0.014793,-0.067737
WinRatio,0.093182,-0.491599,0.073625,1.000000,-0.218273,0.133133,0.527085,0.527681,-0.061393,0.532667,0.518147,0.529943,0.296620,-0.768662,-0.623153,-0.726418
FirstD1Season,-0.287709,0.085927,-0.281736,-0.218273,1.000000,0.019527,-0.629420,-0.628600,-0.566708,-0.607875,-0.651119,-0.617230,-0.186541,0.280447,0.119842,0.298387
LastD1Season,0.165160,-0.056504,0.162693,0.133133,0.019527,1.000000,0.199466,0.200371,-0.082981,0.216299,0.178025,0.211726,0.221572,-0.173076,0.019211,0.008428
Rating_Mean,0.223193,-0.340080,0.198064,0.527085,-0.629420,0.199466,1.000000,0.999958,0.178308,0.998491,0.997882,0.998178,0.483841,-0.676756,-0.454850,-0.657934
Rating_Median,0.223157,-0.340599,0.197989,0.527681,-0.628600,0.200371,0.999958,1.000000,0.176076,0.998577,0.997554,0.998130,0.484838,-0.677590,-0.454266,-0.658368
Rating_Std,0.145506,0.087409,0.150966,-0.061393,-0.566708,-0.082981,0.178308,0.176076,1.000000,0.139042,0.223419,0.157280,-0.118812,0.087435,0.144766,0.056128
Rating_Min,0.220241,-0.348050,0.194694,0.532667,-0.607875,0.216299,0.998491,0.998577,0.139042,1.000000,0.992973,0.998174,0.500527,-0.684478,-0.454182,-0.663223


In [30]:
corr_w = df_w.corr()
corr_w.style.background_gradient(cmap='coolwarm')

,Games,ScoreDiff,Home,WinRatio,Rating_Mean,Rating_Median,Rating_Std,Rating_Min,Rating_Max,Rating_Last,Rating_Trend,RPIDiff,SeedDiff
Games,1.000000,-0.052559,0.995788,0.171917,0.315022,0.314893,0.186122,0.309035,0.320513,0.312088,0.086426,-0.139042,-0.071757
ScoreDiff,-0.052559,1.000000,-0.035520,-0.519959,-0.386781,-0.386601,0.031382,-0.394118,-0.378975,-0.393132,-0.328005,0.561569,0.373683
Home,0.995788,-0.035520,1.000000,0.152957,0.292200,0.292050,0.188657,0.286037,0.298074,0.289446,0.068079,-0.114114,-0.052892
WinRatio,0.171917,-0.519959,0.152957,1.000000,0.601351,0.600950,0.054567,0.604058,0.597956,0.607055,0.430567,-0.756182,-0.570420
Rating_Mean,0.315022,-0.386781,0.292200,0.601351,1.000000,0.999967,0.245801,0.999073,0.998081,0.997561,0.506668,-0.651757,-0.402788
Rating_Median,0.314893,-0.386601,0.292050,0.600950,0.999967,1.000000,0.243418,0.999111,0.997775,0.997290,0.506094,-0.651292,-0.401846
Rating_Std,0.186122,0.031382,0.188657,0.054567,0.245801,0.243418,1.000000,0.219347,0.287163,0.253683,-0.044652,0.008623,0.163199
Rating_Min,0.309035,-0.394118,0.286037,0.604058,0.999073,0.999111,0.219347,1.000000,0.994958,0.996941,0.517314,-0.657764,-0.401349
Rating_Max,0.320513,-0.378975,0.298074,0.597956,0.998081,0.997775,0.287163,0.994958,1.000000,0.997111,0.499393,-0.644992,-0.404898
Rating_Last,0.312088,-0.393132,0.289446,0.607055,0.997561,0.997290,0.253683,0.996941,0.997111,1.000000,0.539717,-0.660973,-0.410743


In [31]:
corr_m = df_m.corr()['WinRatio'].sort_values(ascending=False)
high_corr_m = corr_m[[abs(corr_m) > 0.1 for corr_m in corr_m]]

corr_w = df_w.corr()['WinRatio'].sort_values(ascending=False)
high_corr_w = corr_w[[abs(corr_w) > 0.1 for corr_w in corr_w]]

display(high_corr_m)
display(high_corr_w)

WinRatio         1.000000
Rating_Min       0.532667
Rating_Last      0.529943
Rating_Median    0.527681
Rating_Mean      0.527085
Rating_Max       0.518147
Rating_Trend     0.296620
LastD1Season     0.133133
FirstD1Season   -0.218273
ScoreDiff       -0.491599
SeedDiff        -0.623153
RankingsDiff    -0.726418
RPIDiff         -0.768662
Name: WinRatio, dtype: float64

WinRatio         1.000000
Rating_Last      0.607055
Rating_Min       0.604058
Rating_Mean      0.601351
Rating_Median    0.600950
Rating_Max       0.597956
Rating_Trend     0.430567
Games            0.171917
Home             0.152957
ScoreDiff       -0.519959
SeedDiff        -0.570420
RPIDiff         -0.756182
Name: WinRatio, dtype: float64

# Training

In [32]:
def score_dataset(lgbm_params, X, y):
    reg   = lgb.LGBMRegressor(**lgbm_params)
    score = cross_val_score(reg, X, y)
    score = -1 * score.mean() + score.std()

    return score

def objective(trial, X, y):
    params = {
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves': trial.suggest_int('num_leaves', 5, 31),
        'n_estimators': trial.suggest_int('n_estimators', 1, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 300),
        'device_type': device(),
        'verbose': -1
    }

    return score_dataset(params, X, y)

def study(X, y):
    study = op.create_study()
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100, n_jobs=-1, show_progress_bar=True)

    return study.best_params

In [33]:
def build_x_y(df):
    target_column = 'WinRatio'
    feature_columns = df.columns.tolist()
    feature_columns.remove(target_column)
    
    return df[feature_columns], df[target_column]

In [34]:
X_m, y_m = build_x_y(df_m)
X_w, y_w = build_x_y(df_w)

In [35]:
params_m = study(X_m, y_m)
params_w = study(X_w, y_w)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [36]:
def accuracy(X, y, params):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    reg_test = lgb.LGBMRegressor(**params)
    reg_test.fit(X_train, y_train)

    print('LightGBM Model accuracy score: {0:0.4f}'.format(reg_test.score(X_test, y_test)))
    print('LightGBM Model accuracy score [train]: {0:0.4f}'.format(reg_test.score(X_train, y_train)))

In [37]:
accuracy(X_m, y_m, params_m)
accuracy(X_w, y_w, params_w)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014188 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3258
[LightGBM] [Info] Number of data points in the train set: 99754, number of used features: 15
[LightGBM] [Info] Start training from score 0.435861
LightGBM Model accuracy score: 0.9269
LightGBM Model accuracy score [train]: 0.9259
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022960 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2954
[LightGBM] [Info] Number of data points in the train set: 98700, number of used features: 12
[LightGBM] [Info] Start training from score 0.408500
LightGBM Model accuracy score: 0.9417
LightGBM Model accuracy score [train]: 0.9443


# Prediction

In [38]:
def build_wins(X, y, params):
    reg = lgb.LGBMRegressor(**params)
    reg.fit(X, y)

    wins = X
    wins['WinRatio'] = reg.predict(X)
    wins = wins[['WinRatio']]

    return wins

In [39]:
wins_m = build_wins(X_m, y_m, params_m)
wins_w = build_wins(X_w, y_w, params_w)

display(wins_m)
display(wins_w)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3316
[LightGBM] [Info] Number of data points in the train set: 142506, number of used features: 15
[LightGBM] [Info] Start training from score 0.435858
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032615 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3012
[LightGBM] [Info] Number of data points in the train set: 141000, number of used features: 12
[LightGBM] [Info] Start training from score 0.408371


WinRatio
TeamID OTeamID          
1101   1102     0.923648
       1103     0.370989
       1104     0.370989
       1105     0.370989
       1106     0.370989
...                  ...
1478   1473     0.302443
       1474     0.302443
       1475     0.302443
       1476     0.302443
       1477     0.302443

[142506 rows x 1 columns]

WinRatio
TeamID OTeamID          
3101   3102     0.484672
       3103     0.484672
       3104     0.484672
       3105     0.484672
       3106     0.921278
...                  ...
3478   3473     0.244302
       3474     0.244302
       3475     0.244302
       3476     0.244302
       3477     0.244302

[141000 rows x 1 columns]

In [40]:
def build_slots(gender):
    slots = CSV["{}NCAATourneySlots".format(gender)]
    slots = slots[slots['Season'] == 2023]
    slots = slots[slots['Slot'].str.contains('R')] 

    return slots

In [41]:
slots_m = build_slots('M')
slots_w = build_slots('W')

display(slots_m)
display(slots_w)

,Season,Slot,StrongSeed,WeakSeed
2385,2023,R1W1,W01,W16
2386,2023,R1W2,W02,W15
2387,2023,R1W3,W03,W14
2388,2023,R1W4,W04,W13
2389,2023,R1W5,W05,W12
...,...,...,...,...
2443,2023,R4Y1,R3Y1,R3Y2
2444,2023,R4Z1,R3Z1,R3Z2
2445,2023,R5WX,R4W1,R4X1
2446,2023,R5YZ,R4Y1,R4Z1


,Season,Slot,StrongSeed,WeakSeed
1579,2023,R1W1,W01,W16
1580,2023,R1W2,W02,W15
1581,2023,R1W3,W03,W14
1582,2023,R1W4,W04,W13
1583,2023,R1W5,W05,W12
...,...,...,...,...
1637,2023,R4Y1,R3Y1,R3Y2
1638,2023,R4Z1,R3Z1,R3Z2
1639,2023,R5WX,R4W1,R4X1
1640,2023,R5YZ,R4Y1,R4Z1


In [42]:
def build_seeds_2024():
    seeds_2024 = CSV['2024_tourney_seeds']

    return seeds_2024[seeds_2024['Tournament'] == 'M'], seeds_2024[seeds_2024['Tournament'] == 'W']

In [43]:
seeds_2024_m, seeds_2024_w = build_seeds_2024()

display(seeds_2024_m)
display(seeds_2024_w)

,Tournament,Seed,TeamID
0,M,W01,1345
1,M,W02,1266
2,M,W03,1243
3,M,W04,1397
4,M,W05,1181
...,...,...,...
59,M,Z12,1433
60,M,Z13,1233
61,M,Z14,1213
62,M,Z15,1421


,Tournament,Seed,TeamID
64,W,W01,3376
65,W,W02,3268
66,W,W03,3323
67,W,W04,3417
68,W,W05,3328
...,...,...,...
123,W,Z12,3405
124,W,Z13,3387
125,W,Z14,3241
126,W,Z15,3436


In [44]:
def prepare_data(seeds):
    seed_dict = seeds.set_index('Seed')['TeamID'].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}

    return seed_dict, inverted_seed_dict


def simulate(round_slots, seeds, inverted_seeds, wins):
    '''
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - wins (DF): DF that includes wins prediction per matchup.
    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    '''
    winners = []
    slots = []

    for slot, strong, weak in zip(round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed):
        team_1, team_2 = seeds[strong], seeds[weak]

        team_1_prob = wins.loc[team_1, team_2].WinRatio
        winner = np.random.choice([team_1, team_2], p=[team_1_prob, 1 - team_1_prob])

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    return [inverted_seeds[w] for w in winners], slots


def run_simulation(seeds, round_slots, wins, brackets):
    '''
    Runs a simulation of bracket tournaments.

    Parameters:
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - wins (DF): DF that includes wins prediction per matchup.
    - brackets (int): Number of brackets to simulate.
    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    '''
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict = prepare_data(seeds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, wins)
        
        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({'Bracket': bracket, 'Slot': slots, 'Team': results})

    return result_df

In [45]:
n_brackets = 100000
result_m = run_simulation(seeds_2024_m, slots_m, wins_m, n_brackets)
result_m.insert(0, 'Tournament', 'M')
result_w = run_simulation(seeds_2024_w, slots_w, wins_w, n_brackets)
result_w.insert(0, 'Tournament', 'W')

100%|██████████| 100000/100000 [18:11<00:00, 91.59it/s]


In [46]:
submission = pd.concat([result_m, result_w])
submission.reset_index(inplace=True, drop=True)
submission.index.names = ['RowId']

submission

,Tournament,Bracket,Slot,Team
RowId,,,,
0,M,1,R1W1,W01
1,M,1,R1W2,W15
2,M,1,R1W3,W03
3,M,1,R1W4,W04
4,M,1,R1W5,W05
...,...,...,...,...
12599995,W,100000,R4Y1,Y01
12599996,W,100000,R4Z1,Z04
12599997,W,100000,R5WX,X05


In [47]:
submission.to_csv('submission.csv')

# Resources
- General guidance: https://www.kaggle.com/code/toshimelonhead/ncaa-march-madness-sabermetric-spin-v2 by _toshimelonhead_.
- Simulation based on: https://www.kaggle.com/code/lennarthaupts/simulate-n-brackets by _Lennart Haupts_.
- ELO rating calculation based on: https://www.kaggle.com/code/lennarthaupts/calculate-elo-ratings by _Lennart Haupts_.